# Read and Validate [LORENE](https://lorene.obspm.fr/) Initial Data

## Author(s): Zach Etienne & Leo Werneck

In [ ]:
import os, sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

if not os.path.exists(os.path.join("lorene_standalone", "Lorene", "Lib", "liblorenef77_g.a")):
    print("""# Error: Lorene hasn't been compiled yet. Please run the following from nrpytutorial/:
cd in_progress  # Make sure you're in the nrpytutorial root directory.
git clone https://bitbucket.org/zach_etienne/lorene_standalone.git
cd lorene_standalone/
# For "Lorene1": wget http://astro.phys.wvu.edu/zetienne/resu.d
# Lorene2 (latest):
wget --no-check-certificate https://ccrgpages.rit.edu/~jfaber/BNSID/Data/simple_polytrope/gam2.5/gam2.5_1.4_1.4/gam2.5_1.4_1.4_hr/gam2.5_1.4_1.4_hr_50/resu_5.000000e+01_1.520000e+00_1.520000e+00.d -O resu.d
cd Lorene/
HOME_LORENE=`pwd` make -j20
""")
    sys.exit(1)
if not os.path.exists(os.path.join("lorene_standalone", "resu.d")):
    print("""# Error: resu.d not found.
# Be sure to go into nrpytutorial
# and run:
cd in_progress/lorene_standalone
# For "Lorene1": wget http://astro.phys.wvu.edu/zetienne/resu.d
# Lorene2 (latest):
wget --no-check-certificate https://ccrgpages.rit.edu/~jfaber/BNSID/Data/simple_polytrope/gam2.5/gam2.5_1.4_1.4/gam2.5_1.4_1.4_hr/gam2.5_1.4_1.4_hr_50/resu_5.000000e+01_1.520000e+00_1.520000e+00.d -O resu.d
""")
    sys.exit(1)

In [ ]:
import shutil
import reference_metric as rfm
import NRPy_param_funcs as par
import cmdline_helper as cmd
from outputC import add_to_Cfunction_dict

# Step P1: Create C code output directory:
Ccodesdir = os.path.join("lorene_standalone", "interpolator_new_way")
# Step P1.a: First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Step P1.b: Then create a fresh directory
cmd.mkdir(Ccodesdir)

# Step P2: Set basic NRPy+ parameters
CoordSystem = "Spherical"
par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)

enable_rfm_precompute = True
par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")

# Step P3: Set up reference metric quantities
rfm.reference_metric()

# Step P4: Disable FD_functions
FD_functions_enable = False

# STep P5: Disable SIMD
SIMD_enable = False

if enable_rfm_precompute:
    cmd.mkdir(os.path.join(Ccodesdir, "rfm_files/"))
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
    par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", os.path.join(Ccodesdir, "rfm_files/"))
    
if SIMD_enable:
    cmd.mkdir(os.path.join(Ccodesdir,"SIMD"))
    shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(Ccodesdir,"SIMD/"))
    
# Dummy parameter
thismodule = "Lorene_ID"
wavespeed  = par.Cparameters("REAL",thismodule,"wavespeed",1.0)

In [ ]:
domain_size     = 64
sinh_width      = 0.2
sinhv2_const_dr = 0.05
SymTP_bScale    = 1.0
CFL_FACTOR      = 0.5

# Step 1.c.i: Set free_parameters.h
with open(os.path.join(Ccodesdir,"free_parameters.h"),"w") as file:
    file.write("""
// Free parameters related to physical system:
params.wavespeed = 1.0;

// Free parameters related to numerical timestep:
REAL CFL_FACTOR = """+str(CFL_FACTOR)+";\n")

# Append to $Ccodesrootdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"),
                                        domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

In [ ]:
# Step 1.c.iv: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

In [ ]:
# Generate C code for converting the Cartesian
# Lorene initial data to the NRPy CoordSystem
import BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear_new_way as AtoBnum
AtoBnum.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear_new_way("Cartesian","ID_Lorene_ADM_quantities",
                                                                       Ccodesdir=Ccodesdir,loopopts="",
                                                                       pass_idx=True)

In [ ]:
import sympy as sp
import grid as gri
import indexedexp as ixp
from outputC import outputC
import GRFFE.equations as GRFFE

# Metric quantities
DIM     = 3
alpha   = sp.Symbol("other_inputs.alp[IDX3S(i0,i1,i2)]",real=True)
betaU   = ixp.zerorank1()
gammaDD = ixp.zerorank2()
for i in range(DIM):
    betaU[i] = sp.Symbol("other_inputs.beta"+chr(ord('x')+i)+"[IDX3S(i0,i1,i2)]",real=True)
    for j in range(i,DIM):
        gammaDD[i][j] = sp.Symbol("other_inputs.g"+chr(ord('x')+i)+chr(ord('x')+j)+"[IDX3S(i0,i1,i2)]",real=True)
        
# Hydro quantities
rho_b    = gri.register_gridfunctions("AUX","rhob")
u4U      = ixp.register_gridfunctions_for_single_rank1("AUX","u4U",DIM=4)
smallb4U = ixp.register_gridfunctions_for_single_rank1("AUX","smallb4U",DIM=4)
GRFFE.compute_smallbsquared(gammaDD, betaU, alpha, smallb4U)

M_PI,K,Gamma = sp.symbols("M_PI K Gamma",real=True)

# Pressure
P = K * rho_b**Gamma

# Specific internal energy
epsilon = P/rho_b/(Gamma-1)

import GRMHD.equations as GRMHDeqs
GRMHDeqs.compute_GRMHD_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U, smallb4U, GRFFE.smallbsquared)

# Compute rho
n4D    = ixp.zerorank1(DIM=4)
n4D[0] = -alpha
rho    = sp.sympify(0)
for mu in range(4):
    for nu in range(4):
        rho += n4D[mu] * n4D[nu] * GRMHDeqs.T4UU[mu][nu]

rhoADM = gri.register_gridfunctions("AUX","rhoADM")

In [ ]:
def add_to_Cfunction_dict_Hamiltonian_constraint_source_term_new_way():
    desc = """(c) 2021 Leo Werneck"""
    includes = ["NRPy_basic_defines.h"]
    prefunc  = ""
    c_type   = "void"
    name     = "Hamiltonian_constraint_source_term"
    params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                  const REAL K, const REAL Gamma, ID_inputs other_inputs,
                  REAL *restrict aux_gfs"""
    body     = """
    const int idx        = IDX3S(i0,i1,i2);
    const REAL rhob      = aux_gfs[IDX4ptS(RHOBGF,idx)];
    const REAL u4U0      = aux_gfs[IDX4ptS(U4U0GF,idx)];
    const REAL u4U1      = aux_gfs[IDX4ptS(U4U1GF,idx)];
    const REAL u4U2      = aux_gfs[IDX4ptS(U4U2GF,idx)];
    const REAL u4U3      = aux_gfs[IDX4ptS(U4U3GF,idx)];
    const REAL smallb4U0 = aux_gfs[IDX4ptS(SMALLB4U0GF,idx)];
    const REAL smallb4U1 = aux_gfs[IDX4ptS(SMALLB4U1GF,idx)];
    const REAL smallb4U2 = aux_gfs[IDX4ptS(SMALLB4U2GF,idx)];
    const REAL smallb4U3 = aux_gfs[IDX4ptS(SMALLB4U3GF,idx)];
    """+outputC(rho,gri.gfaccess("aux_gfs", "rhoADM"), "returnstring",
                params="outCverbose=False")
    loopopts = "InteriorPoints"
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,loopopts=loopopts)

In [ ]:
import finite_difference as fin
from outputC import lhrh

cmd.mkdir(os.path.join(Ccodesdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(Ccodesdir,"rfm_files/"))

# Now register the Hamiltonian as a gridfunction.
# H = gri.register_gridfunctions("AUX","H")
# Then define the Hamiltonian constraint and output the optimized C code
import BSSN.BSSN_constraints as bssncon
import BSSN.BSSN_stress_energy_source_terms as Bsest
bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)

# Reset to False to disable rfm_precompute
par.set_parval_from_str("reference_metric::enable_rfm_precompute","False")
rfm.ref_metric__hatted_quantities()

def add_to_Cfunction_dict_Hamiltonian_constraint_no_source_term_new_way():
    desc = """(c) 2021 Leo Werneck"""
    includes = ["NRPy_basic_defines.h"]
    prefunc  = ""
    c_type   = "void"
    name     = "Hamiltonian_constraint_no_source_term"
    params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                  REAL *restrict in_gfs, REAL *restrict aux_gfs"""
    body     = fin.FD_outputC("returnstring",lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                              params="outCverbose=False")
    loopopts = "InteriorPoints,Enable_rfm_precompute"
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,loopopts=loopopts)

In [ ]:
from outputC import outC_NRPy_basic_defines_h_dict
def generate_supplementary_dict():
    # First register C functions needed by grid

    # Then set up the dictionary entry for grid in NRPy_basic_defines
    supplementary_dict = {}
    supplementary_dict["Lorene_ID"] = r"""
typedef struct of_ID_inputs {
    // This struct is used to store Lorene
    // initial data in Cartesian basis
    REAL *alp,*betax,*betay,*betaz;
    REAL *gxx,*gxy,*gxz,*gyy,*gyz,*gzz;
    REAL *kxx,*kxy,*kxz,*kyy,*kyz,*kzz;    
} ID_inputs;
"""
    return supplementary_dict

In [ ]:
def add_to_Cfunction_dict_ID_Lorene_ADM_quantities_new_way():
    desc = """(c) 2021 Leo Werneck"""
    includes = ["NRPy_basic_defines.h"]
    prefunc  = ""
    c_type   = "void"
    name     = "ID_Lorene_ADM_quantities"
    params   = """REAL xyz_or_rthph[3], int idx, ID_inputs other_inputs,
                  REAL *gammaSphorCartDD00,REAL *gammaSphorCartDD01,REAL *gammaSphorCartDD02,
                  REAL *gammaSphorCartDD11,REAL *gammaSphorCartDD12,REAL *gammaSphorCartDD22,
                  REAL *KSphorCartDD00,REAL *KSphorCartDD01,REAL *KSphorCartDD02,
                  REAL *KSphorCartDD11,REAL *KSphorCartDD12,REAL *KSphorCartDD22,
                  REAL *alphaSphorCart,REAL *betaSphorCartU0,REAL *betaSphorCartU1,REAL *betaSphorCartU2,
                  REAL *BSphorCartU0,REAL *BSphorCartU1,REAL *BSphorCartU2"""
    body     = r"""    
    // Set gamma_{ij}
    *gammaSphorCartDD00 = other_inputs.gxx[idx];
    *gammaSphorCartDD01 = other_inputs.gxy[idx];
    *gammaSphorCartDD02 = other_inputs.gxz[idx];
    *gammaSphorCartDD11 = other_inputs.gyy[idx];
    *gammaSphorCartDD12 = other_inputs.gyz[idx];
    *gammaSphorCartDD22 = other_inputs.gzz[idx];

    // Set K_{ij}
    *KSphorCartDD00 = other_inputs.kxx[idx];
    *KSphorCartDD01 = other_inputs.kxy[idx];
    *KSphorCartDD02 = other_inputs.kxz[idx];
    *KSphorCartDD11 = other_inputs.kyy[idx];
    *KSphorCartDD12 = other_inputs.kyz[idx];
    *KSphorCartDD22 = other_inputs.kzz[idx];

    // Set lapse function
    *alphaSphorCart = other_inputs.alp[idx];

    // Initialize beta^{i} and B^{i} to zero
    *betaSphorCartU0 = 0.0;
    *betaSphorCartU1 = 0.0;
    *betaSphorCartU2 = 0.0;
    *BSphorCartU0    = 0.0;
    *BSphorCartU1    = 0.0;
    *BSphorCartU2    = 0.0;
"""
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,enableCparameters=False)

In [ ]:
def add_to_Cfunction_dict_main__interpolator_new_way():
    desc = """
(c) 2009 Erik Schnetter
(c) 2010 Frank Loeffler
Edits by Z Etienne 2021
"""

    includes = ["<cstdio>", "<cstring>", "<vector>", "<ios>", "<iostream>",
                "assert.h", "stdlib.h", "bin_ns.h", "unites.h",
                "NRPy_basic_defines.h","NRPy_function_prototypes.h"]

    prefunc = r"""
using namespace std;

// define namespace here for old versions of Lorene that don't do so
namespace Lorene {}
using namespace Lorene;

#define CCTK_EQUALS(a,b) (strcmp((a), (b)) == 0)
#define CCTK_Equals(a,b) (strcmp((a), (b)) == 0)
char initial_shift[200];
char initial_lapse[200];
char initial_dtlapse[200];
char initial_dtshift[200];
char initial_data[200];
"""

    c_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"
    body = r"""
  sprintf(initial_lapse,"Meudon_Bin_NS");
  sprintf(initial_dtlapse,"zero");
  sprintf(initial_shift,  "zero");
  sprintf(initial_dtshift,"zero");
  sprintf(initial_data, "Meudon_Bin_NS");

  if(argc != 3) {
    fprintf(stderr,"Error, incorrect usage. Usage: ./standalone_interpolator [filename (resu.d)] [number of dest points]\n");
    exit(1);
  }

  printf ("Setting up LORENE Bin_NS initial data\n");

  // Meudon data are distributed in SI units (MKSA).  Here are some
  // conversion factors.
  // Be aware: these are the constants Lorene uses. They do differ from other
  // conventions, but they gave the best results in some tests.

  double const c_light  = Unites::c_si;      // speed of light [m/s]
  double const nuc_dens = Unites::rhonuc_si; // Nuclear density as used in Lorene units [kg/m^3]
  double const G_grav   = Unites::g_si;      // gravitational constant [m^3/kg/s^2]
  double const M_sun    = Unites::msol_si;   // solar mass [kg]

  // Cactus units in terms of SI units:
  // (These are derived from M = M_sun, c = G = 1, and using 1/M_sun
  // for the magnetic field)
  double const cactusM = M_sun;
  double const cactusL = cactusM * G_grav / pow(c_light,2);
  double const cactusT = cactusL / c_light;

  // Other quantities in terms of Cactus units
  double const coord_unit = cactusL / 1.0e+3;         // from km (~1.477)
  double const rho_unit   = cactusM / pow(cactusL,3); // from kg/m^3
  int keyerr = 0, anyerr = 0;

  printf ("Setting up coordinates\n");

  int cctk_lsh[3];
  cctk_lsh[0] = atoi(argv[2]);
  cctk_lsh[1] = atoi(argv[3]);
  cctk_lsh[2] = atoi(argv[4]);
  int const npoints = cctk_lsh[0] * cctk_lsh[1] * cctk_lsh[2];

  REAL *xx[3]; // NRPy COORDINATES
  for(int i=0;i<3;i++) xx[i] = (REAL *)malloc(cctk_lsh[i]*sizeof(REAL));
  
  // FIXME: this should be a function!
  for(int i0=0; i0<cctk_lsh[0]; i0++) xx[0][i0] = 1.0;
  for(int i1=0; i1<cctk_lsh[1]; i1++) xx[1][i1] = 1.0;
  for(int i2=0; i2<cctk_lsh[2]; i2++) xx[2][i2] = 1.0;

  vector<double> betax(npoints), betay(npoints), betaz(npoints);
  vector<double> x_Lorene(npoints), y_Lorene(npoints), z_Lorene(npoints); // LORENE COORDINATES, != NRPy COORDINATES

#pragma omp parallel for
  for (int i=0; i<npoints; ++i) {
    x_Lorene[i] = xx[0][i] * coord_unit;
    y_Lorene[i] = xx[1][i] * coord_unit;
    z_Lorene[i] = xx[2][i] * coord_unit;
  }

  // --------------------------------------------------------------
  //   CHECKING FILE NAME EXISTENCE
  // --------------------------------------------------------------
  FILE *file;
  char filename[100];
  sprintf(filename,"%s",argv[1]);
  if ((file = fopen(filename, "r")) != NULL)
    fclose(file);
  else {
    fprintf(stderr,
            "File \"%s\" does not exist. ABORTING\n", filename);
    exit(1);
  }

  printf( "Reading from file \"%s\"\n", filename);

  try {

    // Allocate memory for initial data gridfunctions on NRPy grid
    REAL *id_gfs = (REAL *)malloc(NUM_EVOL_GFS*npoints*sizeof(REAL));
    
    // Declare NRPy parameter struct
    paramstruct params;
#include "set_Cparameters_default.h"
#include "free_parameters.h"
    
    // Declare rfm_struct
    rfm_struct rfmstruct;
    rfm_precompute_rfmstruct_define(&params,xx,&rfmstruct);
    
    const int Nxx_plus_2NGHOSTS0 = cctk_lsh[0] + 2*NGHOSTS;
    const int Nxx_plus_2NGHOSTS1 = cctk_lsh[1] + 2*NGHOSTS;
    const int Nxx_plus_2NGHOSTS2 = cctk_lsh[2] + 2*NGHOSTS;
    
    params.Nxx_plus_2NGHOSTS0 = Nxx_plus_2NGHOSTS0;
    params.Nxx_plus_2NGHOSTS1 = Nxx_plus_2NGHOSTS1;
    params.Nxx_plus_2NGHOSTS2 = Nxx_plus_2NGHOSTS2;

    // Allocate memory for initial data gridfunction on Lorene grid
    ID_inputs other_inputs;
    other_inputs.alp = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gxx = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gxy = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gxz = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gyy = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gyz = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.gzz = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kxx = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kxy = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kxz = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kyy = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kyz = (REAL *)malloc(npoints*sizeof(REAL));
    other_inputs.kzz = (REAL *)malloc(npoints*sizeof(REAL));
    
    Bin_NS bin_ns(npoints, &x_Lorene[0], &y_Lorene[0], &z_Lorene[0], filename);

    printf( "omega [rad/s]:       %g\n", bin_ns.omega);
    printf( "dist [km]:           %g\n", bin_ns.dist);
    printf( "dist_mass [km]:      %g\n", bin_ns.dist_mass);
    printf( "mass1_b [M_sun]:     %g\n", bin_ns.mass1_b);
    printf( "mass2_b [M_sun]:     %g\n", bin_ns.mass2_b);
    printf( "mass_ADM [M_sun]:    %g\n", bin_ns.mass_adm);
    printf( "L_tot [G M_sun^2/c]: %g\n", bin_ns.angu_mom);
    printf( "rad1_x_comp [km]:    %g\n", bin_ns.rad1_x_comp);
    printf( "rad1_y [km]:         %g\n", bin_ns.rad1_y);
    printf( "rad1_z [km]:         %g\n", bin_ns.rad1_z);
    printf( "rad1_x_opp [km]:     %g\n", bin_ns.rad1_x_opp);
    printf( "rad2_x_comp [km]:    %g\n", bin_ns.rad2_x_comp);
    printf( "rad2_y [km]:         %g\n", bin_ns.rad2_y);
    printf( "rad2_z [km]:         %g\n", bin_ns.rad2_z);
    printf( "rad2_x_opp [km]:     %g\n", bin_ns.rad2_x_opp);
    double K = bin_ns.kappa_poly1 * pow((pow(c_light, 6.0) /
                                         ( pow(G_grav, 3.0) * M_sun * M_sun *
                                           nuc_dens )),bin_ns.gamma_poly1-1.);
    printf( "K [ET unit]:         %.15g\n", K);

    assert (bin_ns.np == npoints);

    printf("Filling in Cactus grid points\n");

#pragma omp parallel for
    for(int i=0; i<npoints; ++i) {

      other_inputs.alp[i] = bin_ns.nnn[i];


      //  if(/* Set beta to zero or Lorene */) {
      //    betax[i] = -bin_ns.beta_x[i];
      //    betay[i] = -bin_ns.beta_y[i];
      //    betaz[i] = -bin_ns.beta_z[i];
      //  }

      other_inputs.gxx[i] = bin_ns.g_xx[i];
      other_inputs.gxy[i] = bin_ns.g_xy[i];
      other_inputs.gxz[i] = bin_ns.g_xz[i];
      other_inputs.gyy[i] = bin_ns.g_yy[i];
      other_inputs.gyz[i] = bin_ns.g_yz[i];
      other_inputs.gzz[i] = bin_ns.g_zz[i];

      other_inputs.kxx[i] = bin_ns.k_xx[i] * coord_unit;
      other_inputs.kxy[i] = bin_ns.k_xy[i] * coord_unit;
      other_inputs.kxz[i] = bin_ns.k_xz[i] * coord_unit;
      other_inputs.kyy[i] = bin_ns.k_yy[i] * coord_unit;
      other_inputs.kyz[i] = bin_ns.k_yz[i] * coord_unit;
      other_inputs.kzz[i] = bin_ns.k_zz[i] * coord_unit;

      if(CCTK_EQUALS(initial_data, "Meudon_Bin_NS")) {
        /* IGNORE ALL HYDRO:
           rho[i] = bin_ns.nbar[i] / rho_unit;
           if (!recalculate_eps)
           eps[i] = bin_ns.ener_spec[i];
           // Pressure from EOS_Omni call
           if (CCTK_ActiveTimeLevelsVN(cctkGH, "HydroBase::temperature") > 0 &&
           CCTK_ActiveTimeLevelsVN(cctkGH, "HydroBase::Y_e") > 0)
           {
           EOS_Omni_press(*init_eos_key,recalculate_eps,eos_precision,1,&(rho[i]),&(eps[i]),
           &(temperature[i]),&(Y_e[i]),&(press[i]),&keyerr,&anyerr);
           }
           else
           {
           EOS_Omni_press(*init_eos_key,recalculate_eps,eos_precision,1,&(rho[i]),&(eps[i]),
           NULL,NULL,&(press[i]),&keyerr,&anyerr);
           }

           vel[i          ] = bin_ns.u_euler_x[i];
           vel[i+  npoints] = bin_ns.u_euler_y[i];
           vel[i+2*npoints] = bin_ns.u_euler_z[i];

           // Especially the velocity is set to strange values outside of the
           // matter region, so take care of this in the following way
           if (rho[i] < 1.e-20) {
           rho[i          ] = 1.e-20;
           vel[i          ] = 0.0;
           vel[i+  npoints] = 0.0;
           vel[i+2*npoints] = 0.0;
           eps[i          ] = K * pow(rho[i], bin_ns.gamma_poly1-1.) / (bin_ns.gamma_poly1-1.);
           press[i        ] = K * pow(rho[i], bin_ns.gamma_poly1);
           }
        */
      }

    } // END for i

    printf(" ADM OUTPUT: %e %e %e %e %e %e\n",
           other_inputs.gxx[0],
           other_inputs.gxy[0],
           other_inputs.gxz[0],
           other_inputs.gyy[0],
           other_inputs.gyz[0],
           other_inputs.gzz[0]);

    // Compute BSSN quantities from ADM Cartesian quantities
    ID_BSSN__ALL_BUT_LAMBDAs(&params,xx,other_inputs,id_gfs);
    ID_BSSN_lambdas(         &params,xx,             id_gfs);

    printf("BSSN OUTPUT: %e %e %e %e %e %e\n",
           id_gfs[IDX4ptS(HDD00GF,0)],
           id_gfs[IDX4ptS(HDD01GF,0)],
           id_gfs[IDX4ptS(HDD02GF,0)],
           id_gfs[IDX4ptS(HDD11GF,0)],
           id_gfs[IDX4ptS(HDD12GF,0)],
           id_gfs[IDX4ptS(HDD22GF,0)]);

    {
      // Angular velocity
      double const omega = bin_ns.omega * cactusT;

      // These initial data assume a helical Killing vector field

      if (CCTK_EQUALS(initial_lapse, "Meudon_Bin_NS")) {
        if (CCTK_EQUALS (initial_dtlapse, "Meudon_Bin_NS")) {
          printf ("Error: Nonzero time derivatives of lapse unsupported\n");
          exit(1);
          //set_dt_from_domega (npoints,alp, dtalp, omega);
        } else if (CCTK_EQUALS (initial_dtlapse, "none") or CCTK_EQUALS(initial_dtlapse,"zero")) {
          // do nothing
        } else {
          fprintf(stderr, "internal error\n");
          exit(1);
        }
      }

      if (CCTK_EQUALS(initial_shift, "Meudon_Bin_NS")) {
        if (CCTK_EQUALS (initial_dtshift, "Meudon_Bin_NS")) {
          printf ("Error: Nonzero time derivatives of shift unsupported\n");
          exit(1);
          /*
            set_dt_from_domega (npoints,betax, dtbetax, omega);
            set_dt_from_domega (npoints,betay, dtbetay, omega);
            set_dt_from_domega (npoints,betaz, dtbetaz, omega);*/

        } else if (CCTK_EQUALS (initial_dtshift, "none") or CCTK_EQUALS(initial_dtshift,"zero")) {
          // do nothing
        } else {
          printf("internal error\n");
        }
      }
    }
    for(int i=0;i<3;i++) free(xx[i]);
    free(id_gfs);
    free(other_inputs.alp);
    free(other_inputs.gxx);
    free(other_inputs.gxy);
    free(other_inputs.gxz);
    free(other_inputs.gyy);
    free(other_inputs.gyz);
    free(other_inputs.gzz);
    free(other_inputs.kxx);
    free(other_inputs.kxy);
    free(other_inputs.kxz);
    free(other_inputs.kyy);
    free(other_inputs.kyz);
    free(other_inputs.kzz);

    printf ("Done.\n");
  } catch (ios::failure e) {
    for(int i=0;i<3;i++) free(xx[i]);
    printf("Could not read initial data from file '%s': %s\n", filename, e.what());
  }
"""
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body, enableCparameters=False)

In [ ]:
# add all other functions to Cfunction_dict first!

# Look at Tutorial-Start_to_Finish-Curvilinear_BCs_new_way.ipynb for advice.

# Add ADM ID function to dictionary
add_to_Cfunction_dict_ID_Lorene_ADM_quantities_new_way()

# Add Hamiltonian constraint (no source terms) to dictionary
add_to_Cfunction_dict_Hamiltonian_constraint_no_source_term_new_way()

# Add Hamiltonian constraint (only source terms) to dictionary
add_to_Cfunction_dict_Hamiltonian_constraint_source_term_new_way()

# main function:
add_to_Cfunction_dict_main__interpolator_new_way()

In [ ]:
import outputC as outC

outC.register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
gri.register_C_functions_and_NRPy_basic_defines()  # #define IDX3S(),  etc.
rfm.register_C_functions_and_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=False)

# Output functions for computing all finite-difference stencils.
#   Must be called after defining all functions depending on FD stencils.
if FD_functions_enable:
    fin.output_finite_difference_functions_h(path=Ccodesrootdir)

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesdir, SIMD_enable=SIMD_enable,
                                    supplemental_dict=generate_supplementary_dict())
outC.construct_NRPy_function_prototypes_h(Ccodesdir)

In [ ]:
# import outputC as outC
# print(outC.outC_function_prototype_dict)
# print(outC.outC_function_dict)
cmd.new_C_compile(Ccodesdir, "interpolator_new_way",
                  addl_CFLAGS=["-I../Lorene/Export/C++/Include",
#                                "-I/usr/local/Cellar/fftw/3.3.9_1/include",
#                                "-I/usr/local/Cellar/gsl/2.7/include",
                               "-I../Lorene/C++/Include"],
                  addl_libraries=["-L../Lorene/Lib/",
#                                   "-L/usr/local/Cellar/fftw/3.3.9_1/lib",
#                                   "-L/usr/local/Cellar/gsl/2.7/lib",
                                  "-llorene_export", "-llorene",
                                  "-llorenef77", "-lgfortran", "-lfftw3", "-lgsl",
                                  "-lgslcblas", "-llapack", "-lblas"],
                  CC="g++")

# cmd.new_C_compile(Ccodesrootdir, "CurviBC_Playground", compiler_opt_option="fast") # fastdebug or debug also supported
# orig_working_directory = os.getcwd()
# os.chdir(Ccodesdir)
# cmd.Execute("interpolator_new_way", "../resu.d 100")
# os.chdir(orig_working_directory)

In [ ]:
# import cmdline_helper as cmd
# os.chdir(os.path.join("lorene_standalone","Lorene"))
# cmd.Execute_input_string("g++ -IExport/C++/Include -IC++/Include interpolator.cc -o interpolator -LLib/ -llorene_export -llorene -llorenef77 -lgfortran -lfftw3 -lgsl -lgslcblas -llapack -lblas")
# cmd.Execute("interpolator")
# os.chdir(os.path.join("..",".."))